now we'll experiment the weights and then we'll deploy the model

In [21]:
pip install sdv

In [2]:
import pandas as pd
import numpy as np
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

ModuleNotFoundError: No module named 'sdv'

In [72]:
from sdv.errors import InvalidDataError

In [66]:
model = CTGANSynthesizer.load('/content/ctgan_model.pkl')

In [ ]:
def generate_synthetic_data(model_path, input_data, num_rows=100):
    """Генерация синтетических данных из subject-verb-object"""
    try:
        model = CTGANSynthesizer.load(model_path)

        # 2. creating temporary data with a complete structure
        # first, we create a minimal DataFrame with the necessary columns
        temp_data = pd.DataFrame({
            'subject': input_data['subject'],
            'verb': input_data['verb'],
            'object': input_data['object'],
            # adding dummy numeric columns
            'amount': np.random.uniform(10, 1000, size=len(input_data)),
            'time': pd.to_datetime('2023-01-01') + pd.to_timedelta(np.random.randint(0, 365, len(input_data)), 'days'),
            # adding all possible one-hot speakers
            'verb_bought': (input_data['verb'] == 'bought').astype(int),
            'verb_returned': (input_data['verb'] == 'returned').astype(int),
            'verb_ordered': (input_data['verb'] == 'ordered').astype(int),
            'verb_paid': (input_data['verb'] == 'paid').astype(int),
            'verb_sold': (input_data['verb'] == 'sold').astype(int),
            'object_laptop': (input_data['object'] == 'laptop').astype(int),
            'object_phone': (input_data['object'] == 'phone').astype(int),
            'object_book': (input_data['object'] == 'book').astype(int),
            'object_product': (input_data['object'] == 'product').astype(int),
            'object_service': (input_data['object'] == 'service').astype(int),
            # adding dummy mask columns
            'mask': 1,
            'mask_amount': 1,
            'mask_time': 1,
            'synthetic_id': np.arange(len(input_data))
        })

        # 3. adding possible missing columns
        expected_columns = [
            'amount_norm', 'unix_time_norm', 'object_accessory',
            'verb_cancelled', 'user_id'
        ]
        for col in expected_columns:
            if col not in temp_data.columns:
                temp_data[col] = 0 if not col.endswith('_norm') else np.random.rand(len(input_data))

        # 4. synthetic data generation
        synthetic_data = model.sample(num_rows=num_rows)

        # 5. we extract only the necessary columns
        result = synthetic_data[['subject', 'verb', 'object']].copy()

        # 6. adjusting verb and object based on one-hot columns
        for col in synthetic_data.columns:
            if col.startswith('verb_') and synthetic_data[col].sum() > 0:
                result.loc[synthetic_data[col] == 1, 'verb'] = col.replace('verb_', '')
            if col.startswith('object_') and synthetic_data[col].sum() > 0:
                result.loc[synthetic_data[col] == 1, 'object'] = col.replace('object_', '')

        return result

    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")
        print("Попробуем дообучить модель...")

        # creating metadata for further training
        metadata = SingleTableMetadata()
        metadata.detect_from_dataframe(temp_data)

        # further training of the model
        model = CTGANSynthesizer(metadata, epochs=10)
        model.fit(temp_data)

        # re-generation
        synthetic_data = model.sample(num_rows=num_rows)
        return synthetic_data[['subject', 'verb', 'object']]

In [ ]:
if __name__ == "__main__":
    # input data
    input_df = pd.DataFrame({
        'subject': ['customer1', 'customer2', 'customer3'],
        'verb': ['bought', 'returned', 'ordered'],
        'object': ['laptop', 'book', 'phone']
    })

    # data gen
    try:
        synthetic_df = generate_synthetic_data(
            model_path='/content/ctgan_model.pkl',
            input_data=input_df,
            num_rows=50
        )

        print("Generated data:")
        print(synthetic_df.head())

        synthetic_df.to_csv('synthetic_results.csv', index=False)
        print("The results are saved in synthetic_results.csv")

    except Exception as e:
        print(f"Critical error: {str(e)}")

Произошла ошибка: "['subject'] not in index"
Попробуем дообучить модель...


/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


Сгенерированные данные:
     subject      verb  object
0  customer2    bought    book
1  customer1  returned    book
2  customer3   ordered  laptop
3  customer2  returned   phone
4  customer2  returned   phone
Результаты сохранены в synthetic_results.csv


data

In [4]:
input_df = pd.DataFrame({
        'subject': ['customer1', 'customer2', 'customer3'],
        'verb': ['bought', 'returned', 'ordered'],
        'object': ['laptop', 'book', 'phone']
    })

In [5]:
input_data=input_df

In [ ]:
temp_data = pd.DataFrame({
            'subject': input_data['subject'],
            'verb': input_data['verb'],
            'object': input_data['object'],
            # adding dummy numeric columns
            'amount': np.random.uniform(10, 1000, size=len(input_data)),
            'time': pd.to_datetime('2023-01-01') + pd.to_timedelta(np.random.randint(0, 365, len(input_data)), 'days'),
            # adding all possible one-hot speakers
            'verb_bought': (input_data['verb'] == 'bought').astype(int),
            'verb_returned': (input_data['verb'] == 'returned').astype(int),
            'verb_ordered': (input_data['verb'] == 'ordered').astype(int),
            'verb_paid': (input_data['verb'] == 'paid').astype(int),
            'verb_sold': (input_data['verb'] == 'sold').astype(int),
            'object_laptop': (input_data['object'] == 'laptop').astype(int),
            'object_phone': (input_data['object'] == 'phone').astype(int),
            'object_book': (input_data['object'] == 'book').astype(int),
            'object_product': (input_data['object'] == 'product').astype(int),
            'object_service': (input_data['object'] == 'service').astype(int),
            # adding dummy mask columns
            'mask': 1,
            'mask_amount': 1,
            'mask_time': 1,
            'synthetic_id': np.arange(len(input_data))
        })

In [7]:
temp_data.to_csv('temp_data.csv', index=False)